<a href="https://colab.research.google.com/github/juyingnan/vccf_visualization/blob/main/vitessce/notebook/Vitessce_png_EUI_local.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tifffile vitessce[all] scikit-image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.1/206.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 73.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.1/96.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.6/152.6 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00


In [ ]:
#Imports
from tifffile import imread, TiffWriter
from os.path import join
from vitessce.data_utils import rgb_img_to_ome_tiff
from skimage import io
import importlib.util
from os.path import join

In [ ]:
#Build Vitessce Config
from vitessce import (
    VitessceConfig,
    ViewType as vt,
    CoordinationType as ct,
    FileType as ft,
    MultiImageWrapper,
    OmeTiffWrapper,
    BASE_URL_PLACEHOLDER,
)


In [ ]:
# ATKINSON

from IPython.display import display, HTML
import json
import os
!mkdir -p ./images  # create a directory to store images if it doesn't exist

eui_url = 'https://storagetuzi.blob.core.windows.net/blobtuzi/eui/eui_atkinson.pyramid.ome.tif'
eui_original_file_name = os.path.basename(eui_url)
!curl -L -o ./images/{eui_original_file_name} {eui_url}
dir_path = './images'
full_path = os.path.join(dir_path, eui_original_file_name)
print(full_path)

# LOCAL=1, ONLINE=0
OUTPUT_LEVEL = 1

vc = VitessceConfig(schema_version="1.0.15", name='EUI')
if OUTPUT_LEVEL == 1:
    #LOCAL
    dataset = vc.add_dataset(name='Tissue Blocks in EUI').add_object(
        OmeTiffWrapper(img_path=full_path,name='EUI')
    )
else:
    # ONLINE PREVIEW
    dataset = vc.add_dataset(name='Tissue Blocks in EUI').add_object(
        OmeTiffWrapper(img_url=eui_url,name='EUI')
    )
spatial = vc.add_view(vt.SPATIAL, dataset=dataset)
status = vc.add_view(vt.STATUS, dataset=dataset)
# Try changing the prop below to False
lc = vc.add_view(vt.LAYER_CONTROLLER, dataset=dataset).set_props(disableChannelsIfRgbDetected=False)
# vc.layout(spatial | (lc / status))
vc.layout(spatial)
from IPython.display import display, HTML
web_url = vc.web_app()
display(HTML(f'<a href="{web_url}" target="_blank">View on Vitessce.io</a>'))

ROOT = "eui_atkinson"

PATH_TO_EXPORT_DIRECTORY = os.path.join(ROOT, "data", f"vignette_01")
VIGNETTE_DIR = os.path.join(ROOT, "vignettes", f"vignette_01")
#Export Vitessce config to JSON
os.makedirs(VIGNETTE_DIR, exist_ok=True)
#Export Vitessce config to JSON
os.makedirs(PATH_TO_EXPORT_DIRECTORY, exist_ok=True)
config_dict = vc.export(to="files", base_url=f"{BASE_URL_PLACEHOLDER}/vignette_01", out_dir=PATH_TO_EXPORT_DIRECTORY)
# Use `open` to create a new empty file at ./exported_data/vitessce.json
with open(os.path.join(VIGNETTE_DIR, "vitessce.json"), "w") as f:
    json.dump(config_dict, f)
vignette_md = f"""
    ---
    name: Explore Tissue Data in 3D using the Exploration User Interface
    figures:
        - name: "Visualization"
          file: vitessce.json
    ---

    All HuBMAP tissue datasets used in this study can be spatially explored in their three-dimensional size, position, and rotation in the context of the Human Reference Atlas using the Exploration User Interface (EUI): (https://hubmapconsortium.github.io/hra-registrations/23-hbm-flagship-atkinson/)[https://hubmapconsortium.github.io/hra-registrations/23-hbm-flagship-atkinson/]

"""
with open(os.path.join(VIGNETTE_DIR, "description.md"), "w") as f:
    f.write(vignette_md)

import shutil
from google.colab import files

# Create a zip file
shutil.make_archive('vignette_atkinson', 'zip', ROOT)

# Download the zip file
files.download('vignette_atkinson.zip')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  399k  100  399k    0     0   633k      0 --:--:-- --:--:-- --:--:--  633k
./images/eui_atkinson.pyramid.ome.tif


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# BLUE

from IPython.display import display, HTML
import json
import os
!mkdir -p ./images  # create a directory to store images if it doesn't exist

eui_url = 'https://storagetuzi.blob.core.windows.net/blobtuzi/eui/eui_blue.pyramid.ome.tif'
eui_original_file_name = os.path.basename(eui_url)
!curl -L -o ./images/{eui_original_file_name} {eui_url}
dir_path = './images'
full_path = os.path.join(dir_path, eui_original_file_name)
print(full_path)

# LOCAL=1, ONLINE=0
OUTPUT_LEVEL = 1

vc = VitessceConfig(schema_version="1.0.15", name='EUI')
if OUTPUT_LEVEL == 1:
    #LOCAL
    dataset = vc.add_dataset(name='Tissue Blocks in EUI').add_object(
        OmeTiffWrapper(img_path=full_path,name='EUI')
    )
else:
    # ONLINE PREVIEW
    dataset = vc.add_dataset(name='Tissue Blocks in EUI').add_object(
        OmeTiffWrapper(img_url=eui_url,name='EUI')
    )
spatial = vc.add_view(vt.SPATIAL, dataset=dataset)
status = vc.add_view(vt.STATUS, dataset=dataset)
# Try changing the prop below to False
lc = vc.add_view(vt.LAYER_CONTROLLER, dataset=dataset).set_props(disableChannelsIfRgbDetected=False)
# vc.layout(spatial | (lc / status))
vc.layout(spatial)
from IPython.display import display, HTML
web_url = vc.web_app()
display(HTML(f'<a href="{web_url}" target="_blank">View on Vitessce.io</a>'))

ROOT = "eui_blue"

PATH_TO_EXPORT_DIRECTORY = os.path.join(ROOT, "data", f"vignette_01")
VIGNETTE_DIR = os.path.join(ROOT, "vignettes", f"vignette_01")
#Export Vitessce config to JSON
os.makedirs(VIGNETTE_DIR, exist_ok=True)
#Export Vitessce config to JSON
os.makedirs(PATH_TO_EXPORT_DIRECTORY, exist_ok=True)
config_dict = vc.export(to="files", base_url=f"{BASE_URL_PLACEHOLDER}/vignette_01", out_dir=PATH_TO_EXPORT_DIRECTORY)
# Use `open` to create a new empty file at ./exported_data/vitessce.json
with open(os.path.join(VIGNETTE_DIR, "vitessce.json"), "w") as f:
    json.dump(config_dict, f)
vignette_md = f"""
---
name: Explore Tissue Data in 3D using the Exploration User Interface
figures:
    - name: "Visualization"
      file: vitessce.json
---

All HuBMAP tissue datasets used in this study can be spatially explored in their three-dimensional size, position, and rotation in the context of the Human Reference Atlas using the Exploration User Interface (EUI): [https://hubmapconsortium.github.io/hra-registrations/23-hbm-flagship-bluelake/](https://hubmapconsortium.github.io/hra-registrations/23-hbm-flagship-bluelake/)

"""
with open(os.path.join(VIGNETTE_DIR, "description.md"), "w") as f:
    f.write(vignette_md)

import shutil
from google.colab import files

# Create a zip file
shutil.make_archive('vignette_blue', 'zip', ROOT)

# Download the zip file
files.download('vignette_blue.zip')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  189k  100  189k    0     0   431k      0 --:--:-- --:--:-- --:--:--  431k
./images/eui_blue.pyramid.ome.tif


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# SPRAGGINS

from IPython.display import display, HTML
import json
import os
!mkdir -p ./images  # create a directory to store images if it doesn't exist

eui_url = 'https://storagetuzi.blob.core.windows.net/blobtuzi/eui/eui_spraggins.pyramid.ome.tif'
eui_original_file_name = os.path.basename(eui_url)
!curl -L -o ./images/{eui_original_file_name} {eui_url}
dir_path = './images'
full_path = os.path.join(dir_path, eui_original_file_name)
print(full_path)

# LOCAL=1, ONLINE=0
OUTPUT_LEVEL = 1

vc = VitessceConfig(schema_version="1.0.15", name='EUI')
if OUTPUT_LEVEL == 1:
    #LOCAL
    dataset = vc.add_dataset(name='Tissue Blocks in EUI').add_object(
        OmeTiffWrapper(img_path=full_path,name='EUI')
    )
else:
    # ONLINE PREVIEW
    dataset = vc.add_dataset(name='Tissue Blocks in EUI').add_object(
        OmeTiffWrapper(img_url=eui_url,name='EUI')
    )
spatial = vc.add_view(vt.SPATIAL, dataset=dataset)
status = vc.add_view(vt.STATUS, dataset=dataset)
# Try changing the prop below to False
lc = vc.add_view(vt.LAYER_CONTROLLER, dataset=dataset).set_props(disableChannelsIfRgbDetected=False)
# vc.layout(spatial | (lc / status))
vc.layout(spatial)
from IPython.display import display, HTML
web_url = vc.web_app()
display(HTML(f'<a href="{web_url}" target="_blank">View on Vitessce.io</a>'))

ROOT = "eui_spraggins"

PATH_TO_EXPORT_DIRECTORY = os.path.join(ROOT, "data", f"vignette_01")
VIGNETTE_DIR = os.path.join(ROOT, "vignettes", f"vignette_01")
#Export Vitessce config to JSON
os.makedirs(VIGNETTE_DIR, exist_ok=True)
#Export Vitessce config to JSON
os.makedirs(PATH_TO_EXPORT_DIRECTORY, exist_ok=True)
config_dict = vc.export(to="files", base_url=f"{BASE_URL_PLACEHOLDER}/vignette_01", out_dir=PATH_TO_EXPORT_DIRECTORY)
# Use `open` to create a new empty file at ./exported_data/vitessce.json
with open(os.path.join(VIGNETTE_DIR, "vitessce.json"), "w") as f:
    json.dump(config_dict, f)
vignette_md = f"""
    ---
    name: Explore Tissue Data in 3D using the Exploration User Interface
    figures:
        - name: "Visualization"
          file: vitessce.json
    ---

    All HuBMAP tissue datasets used in this study can be spatially explored in their three-dimensional size, position, and rotation in the context of the Human Reference Atlas using the Exploration User Interface (EUI): (https://hubmapconsortium.github.io/hra-registrations/23-hbm-flagship-spraggins/)[https://hubmapconsortium.github.io/hra-registrations/23-hbm-flagship-spraggins/]

"""
with open(os.path.join(VIGNETTE_DIR, "description.md"), "w") as f:
    f.write(vignette_md)

import shutil
from google.colab import files

# Create a zip file
shutil.make_archive('vignette_spraggins', 'zip', ROOT)

# Download the zip file
files.download('vignette_spraggins.zip')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  386k  100  386k    0     0   622k      0 --:--:-- --:--:-- --:--:--  621k
./images/eui_spraggins.pyramid.ome.tif


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>